# Swarm Ion Temperature Estimate Products in VirES

This notebook demonstrates access to the Swarm LP Ion Drift and Effective Mass products ([`EFIxIDM` products](https://earth.esa.int/eogateway/documents/20142/2860886/SLIDEM_Product_Definition.pdf)). The available variables and their mapping to source product variables are provided.

In [1]:
from viresclient import SwarmRequest

# temporary fix for older client version
# won't be necessary after the upcoming release

SwarmRequest.COLLECTIONS["EFI_IDM"] = [
    "SW_PREL_EFI{}IDM_2_".format(x) for x in "ABC"
]

SwarmRequest.PRODUCT_VARIABLES["EFI_IDM"] = [
    "Latitude_GD", "Longitude_GD", "Height_GD",
    "Radius_GC", "Latitude_QD", "MLT_QD",
    "V_sat_nec", "M_i_eff", "M_i_eff_err", "M_i_eff_Flags",
    "M_i_eff_tbt_model", "V_i", "V_i_err", "V_i_Flags",
    "V_i_raw", "N_i", "N_i_err", "N_i_Flags", "A_fp",
    "R_p", "T_e", "Phi_sc",
]

#SERVER_URL = "https://vires.services/ows" # not yet available in production, will be deployed with the upcoming release
SERVER_URL = "https://staging.viresdisc.vires.services/ows"

## EFIxIDM_2_ - Ion Drift, Density and Effective Mass

### Collections

| Collection | Dataset |
|---|---|
|`SW_PREL_EFIAIDM_2_`| Swarm A ion drift, density and effective mass |
|`SW_PREL_EFIBIDM_2_`| Swarm B ion drift, density and effective mass |
|`SW_PREL_EFICIDM_2_`| Swarm C ion drift, density and effective mass |


### Avaiable Variables

Note that the source EFIxIDM_2_ products contain WGS84 geodetic coordinates rather than the ITRF geocentric sperical coordinates commonly used by other Swarm products. Although the differerences between the latitudes are not very large, VirES coverts the geodetic coordinates to the geocentric ones. The original coordinates from the source products are provided as a reference.


| VirES Variable | Source Product Variable | Unit | Dimension | Description | 
|---|---|---|---|---|
| `Timestamp` | `Timestamp` | $$-$$ | scalar | UTC time-stamp|
| `Latitude` | n/a | $$\text{deg}$$ | scalar | ITRF geocentric latitude calculated from the source geodetic coordinates |
| `Longitude` | n/a | $$\text{deg}$$ | scalar | ITRF geocentric longitude calculated from the source geodetic coordinates |
| `Radius` | n/a | $$\text{m}$$ | scalar | ITRF geocentric radius calculated from the source geodetic coordinates |
| `Latitude_GD` | `Latitude` | $$\text{deg}$$ | scalar | Geodetic latitude from the source product |
| `Longitude_GD` | `Longitude` | $$\text{deg}$$ | scalar | Geodetic longitude from the source product |
| `Height_GD` | `Height` | $$\text{m}$$ | scalar | Height above the WGS84 ellipsoind from the source product |
| `Radius_GC` | `Radius` | $$\text{m}$$ | scalar | Geocentric radius from the source products |
| `Latitude_QD` | `QDLatitude` | $$\text{deg}$$ | scalar | Quasi-dipole latitude from the source product |
| `MLT_QD` | `MLT` | $$\text{hour}$$ | scalar | Magnetic local time (QD) from the source product |
| `V_sat_nec` | `V_sat_nec` | $$\text{m}/\text{s}$$ | vector[3] | Satellite velocity in north, east, centre (NEC) reference frame | 
| `M_i_eff` | `M_i_eff` | $$\text{AMU}$$ | scalar | Ion effective mass |
| `M_i_eff_err` | `M_i_eff_err` | $$\text{AMU}$$ | scalar | Ion effective mass uncertainty |
| `M_i_eff_Flags` | `M_i_eff_Flags` | $$-$$ | scalar | Ion effective mass validity flag. |
| `M_i_eff_tbt_model` | `M_i_eff_tbt_model` | $$\text{AMU}$$ | scalar | Ion effective mass from Truhlik et al. (2015) topside empirical model |
| `V_i` | `V_i` | $$\text{m}/\text{s}$$ | scalar | Ion along-track drift |
| `V_i_err` | `V_i_err` | $$\text{m}/\text{s}$$ | scalar | Ion along-track drift uncertainty |
| `V_i_Flags` | `V_i_Flags` | $$-$$ | scalar | Ion along-track drift validity flag |
| `V_i_raw` | `V_i_raw` | $$\text{m}/\text{s}$$ | scalar | Ion along-track drift without high-latitude detrending |
| `N_i` | `N_i` | $$\text{cm}^{-3}$$ | scalar | Ion density |
| `N_i_err` | `N_i_err` | $$\text{cm}^{-3}$$ | scalar | Ion density unertainty |
| `N_i_Flags` | `N_i_Flags` | $$-$$ | scalar | Ion density validity flag |
| `A_fp` | `A_fp` | $$\text{m}^2$$ | scalar | Modified-OML EFI faceplate area |
| `R_p` | `R_p` | $$\text{m}$$ | scalar | Modified-OML Langmuir spherical probe radius |
| `T_e` | `T_e` | $$\text{K}$$ | scalar | Electron temperature |
| `Phi_sc` | `Phi_sc` | $$\text{V}$$ | scalar | Spacecraft floating potential with respect to plasma potential far from satellite |

See the [EFIxIDM_2_ product specification](https://earth.esa.int/eogateway/documents/20142/2860886/SLIDEM_Product_Definition.pdf) for more details.

In [2]:
request = SwarmRequest(SERVER_URL)

request.set_collection("SW_PREL_EFIAIDM_2_")

request.set_products(
    measurements=SwarmRequest.PRODUCT_VARIABLES["EFI_IDM"], # request all dataset variables
    auxiliaries=['QDLat', 'QDLon', 'MLT'], # QD coordinates and MLT calculated by VirES
)

data = request.get_between(
    start_time="2016-01-02T00:00:00Z",
    end_time="2016-01-03T00:00:00Z",
).as_xarray()

print("Requested variables", ", ".join(SwarmRequest.PRODUCT_VARIABLES["EFI_IDM"]))
print("Response:", data)

Processing:    0%|          |  [ Elapsed: 00:00, Remaining: ? ] [1/1] 

Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (40.975MB)

Requested variables Latitude_GD, Longitude_GD, Height_GD, Radius_GC, Latitude_QD, MLT_QD, V_sat_nec, M_i_eff, M_i_eff_err, M_i_eff_Flags, M_i_eff_tbt_model, V_i, V_i_err, V_i_Flags, V_i_raw, N_i, N_i_err, N_i_Flags, A_fp, R_p, T_e, Phi_sc
Response: <xarray.Dataset>
Dimensions:            (Timestamp: 172776, V_sat_nec_dim1: 3)
Coordinates:
  * Timestamp          (Timestamp) datetime64[ns] 2016-01-02T00:00:00.1969999...
Dimensions without coordinates: V_sat_nec_dim1
Data variables: (12/29)
    Spacecraft         (Timestamp) object 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A'
    Latitude_GD        (Timestamp) float64 31.91 31.94 31.98 ... 8.765 8.733
    Latitude_QD        (Timestamp) float64 41.46 41.49 41.52 ... 1.424 1.389
    M_i_eff_Flags      (Timestamp) uint32 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    QDLat              (Timestamp) float64 41.45 41.48 41.52 ... 1.418 1.382
    V_i                (Timestamp) float64 -1e+05 -1e+05 ... -1e+05 -1e+05
    ...                 ...
    R_p        